https://www.frontiersin.org/articles/10.3389/fnhum.2016.00235/full

https://www.mdpi.com/1424-8220/21/19/6570


https://github.com/dokato/connectivipy

La Coherence Dirigée Partielle, ou PDC (Partial Directed Coherence en anglais), est une mesure de la connectivité dans le domaine de la neurophysiologie et de l'analyse du cerveau. Elle est utilisée pour étudier comment différentes régions du cerveau interagissent les unes avec les autres.

La PDC est particulièrement utile pour comprendre comment l'activité électrique ou électroencéphalographique (EEG) est propagée entre différentes régions du cerveau. Voici quelques points clés pour comprendre la PDC :

Connectivité Dirigée : La PDC permet de mesurer la direction de la connectivité entre les régions cérébrales. Autrement dit, elle indique comment l'activité d'une région du cerveau influence l'activité d'une autre région, et vice versa.

Fréquences et Bandes de Fréquences : La PDC peut être calculée pour différentes fréquences ou bandes de fréquences du signal EEG. Cela permet de comprendre comment la connectivité entre les régions cérébrales varie en fonction de la fréquence.

Utilisation : La PDC est largement utilisée dans la recherche en neurosciences pour étudier des phénomènes tels que la communication cérébrale, la synchronisation entre les régions cérébrales, et les changements dans la connectivité en réponse à des tâches spécifiques ou à des pathologies.

Interprétation : Une PDC proche de 1 entre deux régions signifie une forte connectivité dirigée de l'une vers l'autre, tandis qu'une PDC proche de 0 indique une connectivité faible ou nulle.

Applications : La PDC est utilisée dans divers domaines, y compris la recherche sur les troubles neurologiques, l'étude de la cognition humaine, la neurologie clinique, et même dans le domaine de l'analyse de l'activité cérébrale lors de tâches spécifiques telles que la réflexion ou la perception.

En résumé, la PDC est une mesure importante pour comprendre comment différentes parties du cerveau interagissent et communiquent entre elles. Elle est souvent utilisée dans le cadre de la recherche en neurosciences pour mieux comprendre le fonctionnement du cerveau et les altérations qui peuvent survenir dans diverses conditions.

In [24]:
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from scipy.linalg import toeplitz
from scipy.linalg import solve
from scipy.signal import lfilter
from scot.connectivity import Connectivity

import mne
from mne.io import concatenate_raws, read_raw_edf
import glob


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP, SPoC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mne.decoding import (
    SlidingEstimator,
    GeneralizingEstimator,
    Scaler,
    cross_val_multiscore,
    LinearModel,
    get_coef,
    Vectorizer,
    CSP,
)
import numpy as np
from mne.preprocessing import ICA

from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from utils import preprocess_data

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA

from __future__ import print_function

import numpy as np
try:  # new in sklearn 0.19
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
except ImportError:
    from sklearn.lda import LDA
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import scot
import scot.xvschema

In [2]:
files = glob.glob('../../files/S001/*.edf')
'''
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws = []

for i in [5, 9, 13]:
    current_file = files[i]
    r = read_raw_edf(current_file, preload=True, stim_channel='auto')
    events, _ = mne.events_from_annotations(r)
    if i in [5, 9, 13]:
        new_labels_events = {1:'rest', 2:'action_hand', 3:'action_feet'} # action
    new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
    r.set_annotations(new_annot)
    raws.append(r)
    
raw_obj = concatenate_raws(raws)

Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']


In [22]:
raw = raw_obj.copy()

# filters
notch_freq = 60
raw.notch_filter(notch_freq, fir_design='firwin')

low_cutoff = 8
high_cutoff = 40
raw.filter(low_cutoff, high_cutoff, fir_design='firwin')

events, event_dict = mne.events_from_annotations(raw)
print(raw.info)
print(event_dict)
picks = mne.pick_types(raw.info, meg=True, eeg=True, stim=False, eog=False, exclude='bads')
eegbci.standardize(raw)
montage = mne.channels.make_standard_montage('standard_1005')
raw.set_montage(montage)

## ICA
n_components = 10
ica = ICA(n_components=n_components, random_state=97, max_iter=800)
ica.fit(raw)
components_to_excludes, scores = ica.find_bads_eog(raw, ch_name='Fpz')
if components_to_excludes is not None and len(components_to_excludes) > 0:
    ica.exclude = components_to_excludes
    raw = ica.apply(raw)
else:
    print("No components to exclude")

event_id = {'action_hand': 1, 'action_feet': 2}
events, event_dict = mne.events_from_annotations(raw, event_id=event_id)
tmin = -0.5  # Time before event in seconds
tmax = 4.  # Time after event in seconds
epochs = mne.Epochs(raw, events, event_dict, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method


- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1057 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.2s finished


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Used Annotations descriptions: ['action_feet', 'action_hand', 'rest']
<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 8.0 Hz
 lowpass: 40.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>
{'action_feet': 1, 'action_hand': 2, 'rest': 3}
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 10 components
Fitting ICA took 0.9s.
Using EOG channel: Fpz
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequen

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [56]:
epochs.get_data().shape

(45, 64, 721)

In [25]:
ws = scot.Workspace({'model_order': 30}, reducedim=3, fs=epochs.info['sfreq'])

In [26]:
ws.set_data(epochs.get_data(picks=picks), 'MEG', 'raw')

In [33]:
ws.do_mvarica()
# ws.get_connectivity('ffPDC')

IndexError: boolean index did not match indexed array along dimension 0; dimension is 45 but corresponding boolean dimension is 1

In [34]:
events

array([[  672,     0,     2],
       [ 2000,     0,     1],
       [ 3328,     0,     1],
       [ 4656,     0,     2],
       [ 5984,     0,     2],
       [ 7312,     0,     1],
       [ 8640,     0,     1],
       [ 9968,     0,     2],
       [11296,     0,     1],
       [12624,     0,     2],
       [13952,     0,     2],
       [15280,     0,     1],
       [16608,     0,     1],
       [17936,     0,     2],
       [19264,     0,     1],
       [20672,     0,     1],
       [22000,     0,     2],
       [23328,     0,     1],
       [24656,     0,     2],
       [25984,     0,     2],
       [27312,     0,     1],
       [28640,     0,     2],
       [29968,     0,     1],
       [31296,     0,     1],
       [32624,     0,     2],
       [33952,     0,     1],
       [35280,     0,     2],
       [36608,     0,     2],
       [37936,     0,     1],
       [39264,     0,     2],
       [40672,     0,     1],
       [42000,     0,     2],
       [43328,     0,     1],
       [44

In [53]:
from __future__ import print_function

import numpy as np
try:  # new in sklearn 0.19
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
except ImportError:
    from sklearn.lda import LDA
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import scot
import scot.xvschema
import scotdata.motorimagery as midata

raweeg = midata.eeg.T
triggers = np.asarray(midata.triggers, dtype=int)
classes = midata.classes
fs = midata.samplerate
locs = midata.locations
locs = np.asarray(locs, dtype=float)


In [54]:
raweeg.shape, classes.shape, triggers.shape, fs, locs.shape

((45, 187000), (180,), (180,), 100, (45, 3))

In [59]:
triggers

array([  1250,   2229,   3219,   4199,   5227,   6259,   7271,   8261,
         9267,  10272,  11315,  12283,  13328,  14285,  15272,  16264,
        17268,  18261,  19311,  20305,  21321,  22294,  23282,  24301,
        25301,  26296,  27283,  28317,  29304,  30350,  32450,  33442,
        34440,  35466,  36450,  37488,  38477,  39456,  40459,  41500,
        42550,  43528,  44512,  45512,  46547,  47510,  48538,  49511,
        50520,  51525,  52543,  53536,  54583,  55624,  56593,  57588,
        58553,  59564,  60542,  61573,  63650,  64689,  65723,  66748,
        67766,  68784,  69792,  70784,  71768,  72798,  73813,  74825,
        75781,  76759,  77775,  78766,  79743,  80733,  81740,  82765,
        83788,  84810,  85859,  86847,  87874,  88841,  89868,  90826,
        91786,  92742,  94750,  95731,  96711,  97758,  98737,  99781,
       100814, 101825, 102842, 103794, 104831, 105841, 106855, 107839,
       108868, 109844, 110829, 111859, 112862, 113865, 114883, 115890,
      

In [40]:


# Set random seed for repeatable results
np.random.seed(42)


# Switch backend to scikit-learn
scot.backend.activate('sklearn')


# Set up analysis object
#
# We simply choose a VAR model order of 30, and reduction to 4 components.
ws = scot.Workspace({'model_order': 30}, reducedim=4, fs=fs)
freq = np.linspace(0, fs, ws.nfft_)


# Prepare data
#
# Here we cut out segments from 3s to 4s after each trigger. This is right in
# the middle of the motor imagery period.
data = scot.datatools.cut_segments(raweeg, triggers, 3 * fs, 4 * fs)

# Initialize cross-validation
nfolds = 10
kf = KFold(len(triggers))

# LDA requires numeric class labels
cl = np.unique(midata.classes)
classids = np.array([dict(zip(cl, range(len(cl))))[c] for c in midata.classes])

# Perform cross-validation
lda = LDA()
cm = np.zeros((2, 2))
fold = 0
for train, test in kf:
    fold += 1

    # Perform CSPVARICA
    ws.set_data(data[train, :, :], classes[train])
    ws.do_cspvarica()

    # Find optimal regularization parameter for single-trial fitting
    # ws.var_.xvschema = scot.xvschema.singletrial
    # ws.optimize_var()
    ws.var_.delta = 1

    # Single-trial fitting and feature extraction
    features = np.zeros((len(triggers), 32))
    for t in range(len(triggers)):
        print('Fold {:2d}/{:2d}, trial: {:d}   '.format(fold, nfolds, t),
              end='\r')
        ws.set_data(data[t, :, :])
        ws.fit_var()

        con = ws.get_connectivity('ffPDC')

        alpha = np.mean(con[:, :, np.logical_and(7 < freq, freq < 13)], axis=2)
        beta = np.mean(con[:, :, np.logical_and(15 < freq, freq < 25)], axis=2)

        features[t, :] = np.array([alpha, beta]).flatten()

    lda.fit(features[train, :], classids[train])

    acc_train = lda.score(features[train, :], classids[train])
    acc_test = lda.score(features[test, :], classids[test])

    print('Fold {:2d}/{:2d}, '
          'acc train: {:.3f}, '
          'acc test: {:.3f}'.format(fold, nfolds, acc_train, acc_test))

    pred = lda.predict(features[test, :])
    cm += confusion_matrix(classids[test], pred)

print('\nConfusion Matrix:\n', cm)
print('\nTotal Accuracy: {:.3f}'.format(np.sum(np.diag(cm))/np.sum(cm)))


TypeError: 'KFold' object is not iterable